In [3]:
!pip install yt_dlp
!pip install pytube
!pip install moviepy
!pip install requests

  Using cached pytube-15.0.0-py3-none-any.whl.metadata (5.0 kB)
Using cached pytube-15.0.0-py3-none-any.whl (57 kB)
  Using cached moviepy-1.0.3-py3-none-any.whl
  Using cached decorator-4.4.2-py2.py3-none-any.whl.metadata (4.2 kB)
  Using cached proglog-0.1.10-py3-none-any.whl.metadata (639 bytes)
  Using cached imageio_ffmpeg-0.5.1-py3-none-win_amd64.whl.metadata (1.6 kB)
Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
   ---------------------------------------- 0.0/315.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/315.4 kB ? eta -:--:--
   -- ------------------------------------ 20.5/315.4 kB 217.9 kB/s eta 0:00:02
   ----------- ---------------------------- 92.2/315.4 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 315.4/315.4 kB 2.8 MB/s eta 0:00:00
Using cached imageio_ffmpeg-0.5.1-py3-none-win_amd64.whl (22.6 MB)
Using cached proglog-0.1.10-py3-none-any.whl (6.1 kB)
  Attempting uninstall: decorator
    Found existing installat

In [5]:
import os
import requests
import yt_dlp
from moviepy.editor import AudioFileClip

In [14]:
# Specify your desired output directory
OUTPUT_DIR = './content/audio_files'  # Change this to your preferred directory

# Create the output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
# Function to download YouTube audio
def download_youtube_audio(url):
    try:
        
        ydl_opts = {
            'format': 'bestaudio/best',
            'extractaudio': True,  # Download only audio
            'audioformat': 'wav',  # Save as .wav
            'outtmpl': os.path.join(OUTPUT_DIR,'%(title)s.%(ext)s'),  # Save with the title of the video
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            audio_file_path = os.path.join(OUTPUT_DIR,f"{info['title']}.mp3")
            return audio_file_path
    except Exception as e:
        print(f"Error downloading audio: {e}")
        return None

# Function to convert audio to text using Deepgram API
def audio_to_text(audio_file_path, deepgram_api_key):
    try:
        with open(audio_file_path, 'rb') as audio_file:
            response = requests.post(
                'https://api.deepgram.com/v1/listen',
                headers={'Authorization': f'Token {deepgram_api_key}'},
                data=audio_file
            )
        response.raise_for_status()
        return response.json()['channel']['alternatives'][0]['transcript']
    except Exception as e:
        print(f"Error transcribing audio: {e}")
        return None

# Function to generate audio from text using Eleven Labs API
def text_to_audio(text, eleven_labs_api_key):
    try:
        response = requests.post(
            'https://api.elevenlabs.io/v1/text-to-speech',
            headers={
                'Authorization': f'Bearer {eleven_labs_api_key}',
                'Content-Type': 'application/json'
            },
            json={
                'text': text,
                'voice': 'your_voice_id_here'  # Replace with your desired voice ID
            }
        )
        response.raise_for_status()
        return response.json().get('audio_url', 'No audio URL returned')
    except Exception as e:
        print(f"Error generating audio: {e}")
        return None

# Main function to orchestrate the process
def main(youtube_url, deepgram_api_key, eleven_labs_api_key):
    audio_file_path = download_youtube_audio(youtube_url)
    if not audio_file_path:
        return
    
    transcript = audio_to_text(audio_file_path, deepgram_api_key)
    if not transcript:
        return
    
    audio_url = text_to_audio(transcript, eleven_labs_api_key)
    print("Transcript:", transcript)
    print("Generated Audio URL:", audio_url)

# Replace with your actual keys and YouTube URL
DEEPGRAM_API_KEY = '23b5e3ac9960950ab02892c6e00262775d5c6442'
ELEVEN_LABS_API_KEY = 'sk_519eb4b71be5581a81b6da21b7cba06b1504226f30138689'
YOUTUBE_URL = 'https://www.youtube.com/watch?v=dQw4w9WgXcQ'  # Sample URL

# Run the main function
main(YOUTUBE_URL, DEEPGRAM_API_KEY, ELEVEN_LABS_API_KEY)


[youtube] Extracting URL: https://www.youtube.com/watch?v=dQw4w9WgXcQ
[youtube] dQw4w9WgXcQ: Downloading webpage
[youtube] dQw4w9WgXcQ: Downloading ios player API JSON
[youtube] dQw4w9WgXcQ: Downloading web creator player API JSON
[youtube] dQw4w9WgXcQ: Downloading m3u8 information
[info] dQw4w9WgXcQ: Downloading 1 format(s): 251
[download] Destination: content\audio_files\Rick Astley - Never Gonna Give You Up (Official Music Video).webm
[download] 100% of    3.28MiB in 00:00:00 at 3.68MiB/s   
Error transcribing audio: [Errno 2] No such file or directory: './content/audio_files\\Rick Astley - Never Gonna Give You Up (Official Music Video).mp3'
